In [1]:
import psycopg2
import pandas as pd

In [ ]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port="5433")
print("Connection successful")

In [2]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port="5433")
cur = conn.cursor()
query="""
        SELECT * FROM eur_usd_raw ORDER BY datetime DESC;
"""
cur.execute(query)
raw_data = cur.fetchall()
conn.close()

In [3]:
raw_data

[(1,
  datetime.date(2025, 12, 20),
  Decimal('1.17238'),
  Decimal('1.17258'),
  Decimal('1.17072'),
  Decimal('1.17091'))]

In [4]:
df = pd.DataFrame(raw_data, columns=['index', 'datetime', 'open', 'high', 'low', 'close'])
df = df.drop(columns=["index"])
df

,datetime,open,high,low,close
0,2025-12-20,1.17238,1.17258,1.17072,1.17091


In [5]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port="5433")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS eur_usd_features(
        id SERIAL PRIMARY KEY,
        datetime DATE NOT NULL UNIQUE,
        open NUMERIC,
        high NUMERIC,
        low NUMERIC,
        close NUMERIC,
        close_ratio_2 NUMERIC,
        close_ratio_5 NUMERIC,
        close_ratio_60 NUMERIC,
        close_ratio_250 NUMERIC,
        close_ratio_1000 NUMERIC
    );
""")
conn.commit()
conn.close()

In [9]:
import psycopg2.extras as extras

def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ', '.join(list(df.columns))

    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("the dataframe is inserted")
    cursor.close()

In [10]:
df = pd.read_csv("eur_usd_forex_data.csv")
df.head()

,datetime,open,high,low,close
0,2025-12-18,1.17449,1.17496,1.17185,1.17255
1,2025-12-17,1.17582,1.17631,1.17046,1.17447
2,2025-12-16,1.17496,1.18017,1.17363,1.17580
3,2025-12-15,1.17387,1.17691,1.17280,1.17496
4,2025-12-13,1.17410,1.17468,1.17386,1.17402


In [13]:
conn = psycopg2.connect(database="app_db", user="admin", password="admin", host="localhost", port="5433")
execute_values(conn, df, 'eur_usd_raw')

the dataframe is inserted


In [12]:
df

,datetime,open,high,low,close
0,2025-12-18,1.17449,1.17496,1.17185,1.17255
1,2025-12-17,1.17582,1.17631,1.17046,1.17447
2,2025-12-16,1.17496,1.18017,1.17363,1.17580
3,2025-12-15,1.17387,1.17691,1.17280,1.17496
4,2025-12-13,1.17410,1.17468,1.17386,1.17402
...,...,...,...,...,...
4995,2006-10-17,1.25400,1.25660,1.25200,1.25450
4996,2006-10-16,1.25050,1.25430,1.24920,1.25411
4997,2006-10-13,1.25611,1.25789,1.24880,1.25050
4998,2006-10-12,1.25251,1.25671,1.25189,1.25600
